In [65]:
class SmithWaterman:
    def __init__(self, sub=None, w=1):
        if sub is not None:
            self.sub = sub
        else:
            self.sub = lambda a, b: 1 if a == b else -1
        self.w = w
        
    def align(self, a, b):
        n = len(a)
        m = len(b)
        
        H = np.zeros((n+1, m+1))
        dirs = np.empty((n, m))
        
        for i, aa in enumerate(a):
            for j, bb in enumerate(b):
                v1 = H[i, j] + self.sub(aa, bb)
                v2 = (H[:i+1, j+1] - self.w*np.arange(i+1, 0, -1)).max()
                v3 = (H[i+1, :j+1] - self.w*np.arange(j+1, 0, -1)).max()
                scores = [v1, v2, v3, 0]
                m_ = np.argmax(scores)
                dirs[i, j] = m_
                H[i+1, j+1] = scores[m_]
                
        i = np.argmax(H.reshape(-1,))
        j = i%(m+1)
        i //= m+1
        print(i, j)
        res = ""
        while i > 0 and j > 0:
            if dirs[i-1, j-1] == 0:
                res += a[i-1]
                i -= 1
                j -= 1
            elif dirs[i-1, j-1] == 1:
                res += '*'
                i -= 1
            elif dirs[i-1, j-1] == 2:
                res += '*'
                j -= 1
            
                
        return H, dirs, res[::-1]

In [70]:
A = 'AAABBAC'
B = 'ABDBAC'
sw = SmithWaterman()
H, dirs, res = sw.align(A, B)

7 6


In [71]:
res

'AB*BAC'

In [72]:
H

array([[0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 1., 0.],
       [0., 0., 2., 1., 1., 0., 0.],
       [0., 0., 1., 1., 2., 1., 0.],
       [0., 1., 0., 0., 1., 3., 2.],
       [0., 0., 0., 0., 0., 2., 4.]])

In [73]:
dirs

array([[0., 2., 3., 3., 0., 2.],
       [0., 0., 3., 3., 0., 0.],
       [0., 0., 3., 3., 0., 0.],
       [1., 0., 2., 0., 1., 0.],
       [3., 0., 0., 0., 2., 2.],
       [0., 1., 0., 1., 0., 2.],
       [1., 0., 3., 1., 1., 0.]])

# Fitch Margoliash

In [93]:
def fitch_margoliash(dist):
    cluster = []
    tree = {}
    
    n = dist.shape[0]
    n_ = n
    names = [i for i in range(n)]
    while len(cluster) < n - 1:
        i = dist.reshape(-1,).argmin()

        row = i // n_
        col = i % n_
        
        row_name = names[row]
        col_name = names[col]
        
        if isinstance(row_name, int):
            cluster.append(row_name)
            
        if isinstance(col_name, int):
            cluster.append(col_name)
        
        ind = np.delete(np.arange(n_, dtype=int), [row, col])
        dist_row = dist[row, ind].mean()
        dist_col = dist[ind, col].mean()
        
        new_name = (row_name, col_name)
        
        tree[(row_name, new_name)] = (dist_row + dist[row, col] - dist_col) / 2
        tree[(col_name, new_name)] = (dist_col + dist[row, col] - dist_row) / 2
        
        if len(cluster) == n - 1:
            tree[(names[0], new_name)] = (dist_col - dist[row, col] + dist_row) / 2
            break
        
        names.remove(row_name)
        names.remove(col_name)
        names.append(new_name)
        
        dist = np.hstack((dist, np.zeros((n_, 1))))
        dist = np.vstack((dist, np.zeros((1, n_+1))))
        dist[-1, -1] = np.inf
        
        for i in range(n_+1):
            dist[i, -1] = (dist[i, row] + dist[i, col]) / 2
            dist[-1, i] = dist[i, -1]
            
        dist = np.delete(dist, [row, col], axis=1)
        dist = np.delete(dist, [row, col], axis=0)
        
        n_ -= 1
    
    return tree

In [94]:
x = np.array(
    [
        [np.inf, 1, 3, 2], 
        [1, np.inf, 6, 8],
        [3, 6, np.inf, 5],
        [2, 8, 5, np.inf]
    ])

In [95]:
fitch_margoliash(x)

{(0, (0, 1)): -1.75,
 (1, (0, 1)): 2.75,
 (2, (2, (0, 1))): 2.75,
 ((0, 1), (2, (0, 1))): 2.25}